In [1]:
import pandas as pd
import numpy as np

from time import sleep
from tqdm import tqdm
import datetime
import pickle

from InstagramAPI import InstagramAPI

https://dev.to/coderasha/automating-instagram-api-with-python-gain-followers-2ap7

In [2]:
user = pd.read_csv('login.csv', sep=';')

username = user.username.values[0]
password = user.password.values[0]

In [3]:
api = InstagramAPI(username, password)
api.USER_AGENT = 'Instagram 10.34.0 Android (18/4.3; 320dpi; 720x1280; Xiaomi; HM lSW; armani; qcom; en_US)'
api.login()

api.searchUsername(username)
result = api.LastJson
for i in  result['user'].keys():
    print(i)

Request return 429 error!
{'message': 'Please wait a few minutes before you try again.', 'status': 'fail'}
Request return 405 error!
{'message': '', 'status': 'fail'}
Login success!

pk
username
full_name
is_private
profile_pic_url
profile_pic_id
is_verified
has_anonymous_profile_picture
media_count
geo_media_count
follower_count
following_count
following_tag_count
biography
external_url
external_lynx_url
can_boost_post
can_see_organic_insights
show_insights_terms
can_convert_to_business
can_create_sponsor_tags
can_be_tagged_as_sponsor
total_igtv_videos
reel_auto_archive
usertags_count
usertag_review_enabled
has_chaining
hd_profile_pic_versions
hd_profile_pic_url_info
has_placed_orders
show_conversion_edit_entry
allowed_commenter_type
is_video_creator
has_profile_video_feed
eligible_shopping_signup_entrypoints
direct_messaging
fb_page_call_to_action_id
address_street
business_contact_method
category
city_id
city_name
contact_phone_number
is_call_to_action_enabled
latitude
longitude
pub

In [55]:
class InstaBot:

    def __init__(self, username, password):
        self.api = InstagramAPI(username, password) 
        
        api = self.api
        api.USER_AGENT = 'Instagram 10.34.0 Android (18/4.3; 320dpi; 720x1280; Xiaomi; HM lSW; armani; qcom; en_US)'
        api.login()
        
    def get_profile_details(self, usernameID = 1, savefile = ''):
        """
        Function
        savefile - path for file without its name. The name will be /profile_usernameID.csv, sep=\t
        return a df with usernameID info such as nickname, full_name, profile_pic_url, follower_count, following_count and media_count
        """
        api = self.api

        api.login()
        assert isinstance(usernameID, int), 'incorrect type of userID'
        assert isinstance(savefile, str), 'incorrect type of savefile'



        if usernameID == 1:
            api.getSelfUsernameInfo()
        else:
            api.getUsernameInfo(usernameID)

        result = api.LastJson
        username = result['user']['username']
        full_name = result['user']['full_name']
        profile_pic_url = result['user']['profile_pic_url']
        followers = result['user']['follower_count']
        following = result['user']['following_count']
        media_count = result['user']['media_count']
        df_profile = pd.DataFrame(
            {'username':username,
            'full name': full_name,
            'profile picture URL':profile_pic_url,
            'followers':followers,
            'following':following,
            'media count': media_count,
            }, index=[0])
        if savefile == '':
            return df_profile
        else:
            df_profile.to_csv(savefile + '/profile_' + str(usernameID) + '.csv', sep='\t', encoding='utf-8')
            return df_profile
        
    def get_followers(self, username):
        """
        return all followers of 'username'
        """
        api = self.api
        
        api.login()
        api.searchUsername(username)
        r = api.LastJson
        return api.getTotalFollowers(r['user']['pk'])

    def get_followings(self, username):
        """
        return all followings of 'username'
        """
        api = self.api
        
        api.login()
        api.searchUsername(username)
        r = api.LastJson
        return api.getTotalFollowings(r['user']['pk'])
    
    def auto_sub(self, your_followers, cnt=10):
        """
        Autosuber
        """
        assert len(your_followers) >= 1, 'users not found'

        user = np.random.choice(your_followers, 1)[0]
        his_followers = api.getTotalFollowers(user['pk'])

        to_follow = [i['pk'] for i in np.random.choice(his_followers, cnt)]
        your_followers_id = [i['pk'] for i in your_followers]
        to_follow = list(set(to_follow) - set(your_followers_id))

        for user in tqdm(to_follow):
            api.follow(user)
            sleep(np.random.randint(15, 45))


        curr_date =  str(datetime.datetime.now().date()).replace('-', '_')   
        with open('followed_' + curr_date + '.pickle' , 'wb') as f:
            pickle.dump(to_follow, f)

        return to_follow
    
    def auto_unsab(self, username, follower_PATH):
        """
        auto unsuber
        """
        api = self.api

        api.login()
        current_followers = self.get_followers(username)
        current_followers_id = [i['pk'] for i in current_followers]

        with open(follower_PATH, 'rb') as f:
            followers_for_check = pickle.load(f)

        to_unsab = list(set(followers_for_check) - set(current_followers_id))

        for user in tqdm(to_unsab):
            api.unfollow(user)
            sleep(np.random.randint(15, 45))

        return 'Done!'
    
    def give_like(self, *userIDs, cnt_likes=5):
        """
        delay - random number from 15 to 45 seconds
        """
        api = self.api
        for user in tqdm(userIDs):
            api.getUserFeed(user)
            feed = api.LastJson['items']
            feedIDs = [i['pk'] for i in feed]
            random_feed = np.random.choice(feedIDs, cnt_likes)
            for feedID in random_feed:
                api.like(int(feedID))
                sleep(np.random.randint(15, 45))

        return 'all likes are given out'

In [53]:
bot = InstaBot(username, password)

Request return 429 error!
{'message': 'Please wait a few minutes before you try again.', 'status': 'fail'}
Request return 405 error!
{'message': '', 'status': 'fail'}
Login success!



In [54]:
bot.give_like(644755273, 1952213018, cnt_likes=3)

  0%|          | 0/2 [00:00<?, ?it/s]

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>


 50%|█████     | 1/2 [01:40<01:40, 100.64s/it]

<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.int64'>


100%|██████████| 2/2 [02:48<00:00, 90.75s/it] 


'all likes are given out'

In [10]:
followers = bot.get_followers(username)

In [12]:
autosub_followers = bot.auto_sub(followers, cnt=5, delay=15)

KeyError: 'big_list'

In [30]:
PATH = 'followed_2020_02_06.pickle'
bot.auto_unsab('bboyzander', PATH)

100%|██████████| 5/5 [01:17<00:00, 15.57s/it]


'Done!'

In [18]:
followers = bot.get_followers(username)
followings = bot.get_followings(username)

followers_id = [i['pk'] for i in followers]
followings_id = [i['pk'] for i in followings]

not_following_me = list(set(followings_id) - set(followers_id))

TODO
 - [ ] Documentation for all functions
 - [ ] Think about extra functions
 - [ ] Test current functionality
 - [ ] Make telegram bot and run it on the server